<a href="https://colab.research.google.com/github/Reis25/Analytics/blob/master/Nova_abordagem_ILS_INTERATED_LOCAL_SEARCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aluno: Demétrios Reis;

Orientação: **Bruno** e **Rian**;

Descrição do problema:


Iterated Local Search (ILS) aplicado a um problema de roteirização:
- Temos Equipes (carros), cada uma com uma base fixa (ponto inicial).
- Temos Serviços (places) que precisam ser atendidos (visitados, por assim dizer).
- Cada equipe possui:
  *   ✅ Identificador;
  *   ✅ Capacidade Máxima de trabalho;
  *   ✅ Ponto inicial de partida (saída da base, coodenadas);
  *   ❌ Zona de atendimento;
  *   ❌ Habilidades para atendimento;

- Cada Serviço possui:

  *   ✅ Identificador;
  *   ✅ coordenadas;
  *   ✅ Tipo: Emergencial, Suspensão de Fornecimento ou Regulada;
  *   ✅ Se Emergencial:
      * Multa;
      * Abragência;
      * Tempo em espera;
      * Quantidade de clientes interrompidos;   
  *   ✅ Suspensão do Fornecimento:
      * Quantidade de Faturas;
      * Valor total do débito;
  *   ✅ Regulada:
      * Prazo;
      * Compensação (multa por atraso);

- **Restrições**:

  * ✅ Capacidade de atendimento das equipes;
  * ✅ Um serviço não pode ser atribido mais de uma vez;
  * ✅ Um serviço deve ser alocado para apenas uma equipe;


- O ILS procura uma alocação dos serviços às equipes minimizando a distância total.
- Depois, as rotas são desenhadas em um mapa interativo (Folium + OSMnx).


**BackLog de melhorias:**

1. ✅**Iniciar com o guloso: retirar a solução
aleatória (Isso vai fazer a solução convergir de forma mais clara);**
2. ✅**Trazer a pontuação de cada visita para cenários que o numero de serviços > somatorio(numero de equipes)** capacidade


*   A função que fará esse resultado é a de custo e por hora está levando em consideração apenas a distância.


3. ✅**Priorizar a implementação do tempo máximo;**
4. Tantar trazer Matriz de serviço (pontos que podem ser visitados);
5. distancia percorrida até o momento -> Atualizar;
6. ✅ Modificar a classe carro e trazer uma Rota: (Trazer uma rota) como retorno de uma função.
7. ✅ Trazer uma classe "Solução" trazendo um conjunto de soluções (um vetor ordenado com os atributos);
8. ✅ A classe solução terá um conjunto de rotas e cada rota tera um aributo;
9. Pensar numa hierarquia para otimizar a lista ordenada de carro, Nisso ela pode ser implementada como guloso e pode ser implementada dentro do próprio objeto; e fazer os métodos fazendo essa modificação;
10. ✅ Definir parâmetro de posição inicial das equipes e ainda sua posição X<sub> x-1</sub>;
11. ✅ Corrigir a função de soma, de forma que calcule o trajeto total;
```python
_calculate_route_distance()
```
12. ✅ Validar a função de cálculo de distancia adicionando como coodenadas, avaliar se o trajeto são por vias de ruas transitáveis;
13. Melhorar a função de Log e can_reach, evidenciar no Log que as restrições foram saatisfeitas;
14. Pelo dinamismo do topo ele não deve pegar ultima posição roda  e pega esse screenshot até o final;
15. Avaliar implementação em C++ (C Lion);
16. Comparação de performance;
17. Avaliação de Interface;  
18. Fazer apresentação para reunião;
19. Assistir vídeos encaminhandos pela equipe do SIGA;
20. Falar com Rian antes da apresentação, bater cada tópico;


21. Na próxima sprint implementar balanço de carga.

Observações sobre os resultados e metodologia:
* Trazer equipes e soluções como classes.
* Função de restrição restringindo apenas distâncias acima de 50 km;

Observações Técnicas:

- Usa ! antes de um comando para informar que essa não é um comando python e sim um comando diretamente no shell do sistema operacional.



In [3]:
#Intalação de Pacotes:
!pip install OSMnx
!pip install folium
!pip install networkx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.3 MB/s eta 0:00:00


In [4]:
import random
import math
import copy
import time
import folium
import networkx as nx
import osmnx as ox
from folium.plugins import BeautifyIcon, MarkerCluster, HeatMap
from enum import Enum
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import json
import os


# Configurações e Funções auxiliares
FILE_LOG = "rotas.log"

# Registra mensagens no log e exibe no console
def log(msg: str):
    ts = time.strftime("%Y-%m-%d %H:%M:%S")
    with open(FILE_LOG, "a", encoding="utf-8") as f:
        f.write(f"[{ts}] {msg}\n")
    print(msg)

# Inicializa arquivo de log
with open(FILE_LOG, "w", encoding="utf-8") as f:
    f.write("=== LOG DE GERAÇÃO DE ROTAS ===\n")



# Cache global do grafo
_GRAFO_MACEIO = None

def carregar_grafo_maceio(tipo_rede='drive'):
    """
    Carrega o grafo de Maceió uma única vez e mantém em cache.
    """
    global _GRAFO_MACEIO

    if _GRAFO_MACEIO is None:
        log("📥 Baixando grafo de Maceió (primeira vez, pode demorar 30-60s)...")

        # Coordenadas do centro de Maceió
        centro_maceio = (-9.6658, -35.7353)

        # Baixa grafo cobrindo toda área urbana de Maceió (~20km de raio)
        _GRAFO_MACEIO = ox.graph_from_point(
            centro_maceio,
            dist=90000,  # 90km de raio
            network_type=tipo_rede
        )
        log("✅ Grafo de Maceió carregado com sucesso!")

    return _GRAFO_MACEIO


def dist_entre_coordenadas(origem_coords, destino_coords, tipo_rede='drive'):
    """
    Calcula distância entre coordenadas usando rede viária real de Maceió.

    Args:
        origem_coords: Tupla (latitude, longitude) da origem
        destino_coords: Tupla (latitude, longitude) do destino
        tipo_rede: Tipo de rede ('drive', 'walk', 'bike')

    Returns:
        Tupla (distancia_metros, rota, grafo) ou (distancia_haversine, None, None) em fallback
    """
    try:
        # Usa o grafo único de Maceió
        G = carregar_grafo_maceio(tipo_rede)

        # Encontra nós mais próximos (ordem: lon, lat)
        origem_node = ox.distance.nearest_nodes(G, origem_coords[1], origem_coords[0])
        destino_node = ox.distance.nearest_nodes(G, destino_coords[1], destino_coords[0])

        # Calcula rota mais curta
        rota = nx.shortest_path(G, origem_node, destino_node, weight='length')
        distancia_total = nx.shortest_path_length(G, origem_node, destino_node, weight='length')

        return distancia_total, rota, G

    except nx.NetworkXNoPath:
        log(f"⚠️  Não foi possível encontrar caminho entre: {origem_coords} e {destino_coords}")
        log("   Usando distância Haversine como alternativa...")
        return _calcular_haversine(origem_coords, destino_coords), None, None

    except Exception as e:
        log(f"❌ Erro ao calcular distância: {e}")
        log("   Usando distância Haversine como alternativa...")
        return _calcular_haversine(origem_coords, destino_coords), None, None


def _calcular_haversine(origem_coords, destino_coords):
    """Calcula distância em linha reta (Haversine) entre duas coordenadas."""
    R = 6371000  # Raio da Terra em metros
    lat1, lon1 = math.radians(origem_coords[0]), math.radians(origem_coords[1])
    lat2, lon2 = math.radians(destino_coords[0]), math.radians(destino_coords[1])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    return R * c


def limpar_cache_grafo():
    """Limpa o cache do grafo para liberar memória."""
    global _GRAFO_MACEIO
    _GRAFO_MACEIO = None
    log("🧹 Cache do grafo limpo")


In [5]:
# Natureza dos serviços:
class ServiceType(Enum):
    EMERGENCY = "Emergencial"
    SUSPENSION = "Suspensão de Fornecimento"
    REGULATED = "Regulada"

#Níveis de prioridade para ordenação
class ServicePriority(Enum):
    CRITICAL = 1
    HIGH = 2
    MEDIUM = 3
    LOW = 4

#Representa um serviço a ser atendido
class Service:
    def __init__(self,
                 service_id,
                 position,
                 service_type=ServiceType.REGULATED,
                 duration=30, # Melhorar usando UPS.
                 required_skills=None # Passar via banco OFS.
                 ):
        self.service_id = service_id
        self.position = position  # (lat, lon)
        self.service_type = service_type
        self.duration = duration  # minutos
        self.required_skills = required_skills or []

        # Dados específicos por tipo
        self._emergency_data = {}
        self._suspension_data = {}
        self._regulated_data = {}

        # Controle de atendimento
        self.is_attended    = False
        self.assigned_route = None
        self.scheduled_time = None

    def __repr__(self):
        return f"Service({self.service_id}, type={self.service_type.value}, pos={self.position})"

    # Define dados para serviço emergencial
    def set_emergency_data(self, penalty, coverage, waiting_time, affected_customers):

        self.service_type = ServiceType.EMERGENCY
        self._emergency_data = {
            'penalty': penalty,
            'coverage': coverage,
            'waiting_time': waiting_time,
            'affected_customers': affected_customers # CI > 1, trata como coletiva, CI = 1 individual.
        }

   # Define dados para suspensão de fornecimento
    def set_suspension_data(self, num_invoices, total_debt):

        self.service_type = ServiceType.SUSPENSION
        self._suspension_data = {
            'num_invoices': num_invoices, # Quantidade de Faturas
            'total_debt': total_debt      # Valor da dívida.
        }

    # Define dados para serviço regulado -> Só levar em consideração o SLA da demanda.
    def set_regulated_data(self, deadline, compensation_penalty):

        self.service_type = ServiceType.REGULATED

        if isinstance(deadline, int):
            deadline = datetime.now() + timedelta(days=deadline)

        self._regulated_data = {
            'deadline': deadline,
            'compensation_penalty': compensation_penalty
        }

    # Calcula prioridade do serviço -> separa por tipoe põe um score.
    def get_priority(self):

        if self.service_type == ServiceType.EMERGENCY:
            data = self._emergency_data
            score = (data.get('affected_customers', 0) * 2 +
                    data.get('waiting_time', 0) * 0.5 +
                    data.get('penalty', 0) * 0.1)
            if score > 100:
                return ServicePriority.CRITICAL
            elif score > 50:
                return ServicePriority.HIGH
            else:
                return ServicePriority.MEDIUM

        # Serviço comercial regulado.
        elif self.service_type == ServiceType.REGULATED:
            data = self._regulated_data
            if 'deadline' in data:
                days = (data['deadline'] - datetime.now()).days
                if days < 0:
                    return ServicePriority.CRITICAL
                elif days <= 2:
                    return ServicePriority.HIGH
                elif days <= 7:
                    return ServicePriority.MEDIUM
                else:
                    return ServicePriority.LOW
            return ServicePriority.MEDIUM

        else:  # CORTE
            debt = self._suspension_data.get('total_debt', 0)
            if debt > 1000:
                return ServicePriority.HIGH
            elif debt > 500:
                return ServicePriority.MEDIUM
            else:
                return ServicePriority.LOW

  # Calcula custo de NÃO atender o serviço
    def calculate_cost(self):
        if self.service_type == ServiceType.EMERGENCY:
            return self._emergency_data.get('penalty', 0) #Levando em consideração apenas o dinheiro.

        elif self.service_type == ServiceType.REGULATED:
            data = self._regulated_data
            if 'deadline' in data:
                days = (data['deadline'] - datetime.now()).days
                if days < 0:
                    return data.get('compensation_penalty', 0) * abs(days)  # Multa diária
                else:
                    return 0
            return data.get('compensation_penalty', 0)

        elif self.service_type == ServiceType.SUSPENSION:
            return self._suspension_data.get('total_debt', 0) # Se não pagar a empresa deixa de arrecadar

        return 0

    # Atribui serviço a uma rota
    def assign_to_route(self, route):

        self.assigned_route = route
        self.is_attended = True

    # Remove atribuição
    def unassign(self):
        self.assigned_route = None
        self.is_attended = False
        self.scheduled_time = None

    # Less Than: compara se um OBJETO é menor que o outro.
    def __lt__(self, other):
        return self.get_priority().value < other.get_priority().value

    # Eq: Equal define quando dois objetos são considerados iguais.
    def __eq__(self, other):
        return self.service_id == other.service_id

    # Torna cada objeto único, atributo de segurança.
    def __hash__(self):
        return hash(self.service_id)

In [6]:
# Representa uma rota (equipe/veículo)
class Route:
    def __init__(self, route_id, position, capacity=8):
        self.route_id = route_id
        self.position = position  # (lat, lon) da base
        self.capacity = capacity
        self.services = []

    def __repr__(self):
        return f"Route({self.route_id}, cap={self.capacity}, pos={self.position})"

    # Verifica se há capacidade para n serviços |
    def has_capacity_for(self, n_services=1):
        return len(self.services) + n_services <= self.capacity

    #Verifica se pode alcançar o serviço (limite de 50km)
    def can_reach(self, service, max_distance=500):
        distance_info = dist_entre_coordenadas(self.position, service.position)
        if distance_info is None or distance_info[0] is None:
            return False
        return (distance_info[0] / 1000) <= max_distance

    # Calcula distância até o serviço
    def distance_to(self, service):
        distance_info = dist_entre_coordenadas(self.position, service.position)
        if distance_info is None or distance_info[0] is None:
            return float('inf')
        return distance_info[0] / 1000  # retorna em km

In [7]:
# Gestão da solução
# Conjunto solução:
class Solution:
    def __init__(self, routes: List[Route]):
        self.routes = routes
        self.assignment: Dict[Route, List[Service]] = {route: [] for route in routes}

        # Cache de métricas
        self._total_distance = None
        self._total_cost = None
        self._is_valid = None

        # Metadados
        self.creation_time = datetime.now()
        self.iteration_found = 0

    def __repr__(self):
        return f"Solution(routes={len(self.routes)}, services={self.count_assigned_services()}, cost={self.total_distance():.2f})"

    # Atribui um serviço a uma rota
    def assign_service(self, route: Route, service: Service) -> bool:

        if route not in self.routes:
            return False

        if service in self.assignment[route]:
            return False

        if self.is_service_assigned(service):
            return False

        if not route.has_capacity_for(1):
            return False

        self.assignment[route].append(service)
        service.assign_to_route(route)
        self._invalidate_cache()
        return True

    # Remove um serviço de sua rota
    def unassign_service(self, service: Service) -> bool:
        for route, services in self.assignment.items():
            if service in services:
                services.remove(service)
                service.unassign()
                self._invalidate_cache()
                return True
        return False

    # Move um serviço entre rotas
    def move_service(self, service: Service, from_route: Route, to_route: Route) -> bool:
        if from_route not in self.routes or to_route not in self.routes:
            return False

        if service not in self.assignment[from_route]:
            return False
        #avalia se tem capacidade
        if not to_route.has_capacity_for(1):
            return False

        self.assignment[from_route].remove(service)
        self.assignment[to_route].append(service)
        service.assign_to_route(to_route)
        self._invalidate_cache()
        return True

    # Troca dois serviços entre rotas
    def swap_services(self, service1: Service, route1: Route,
                     service2: Service, route2: Route) -> bool:

        if route1 not in self.routes or route2 not in self.routes:
            return False

        if service1 not in self.assignment[route1] or service2 not in self.assignment[route2]:
            return False

        idx1 = self.assignment[route1].index(service1)
        idx2 = self.assignment[route2].index(service2)

        self.assignment[route1][idx1] = service2
        self.assignment[route2][idx2] = service1

        service1.assign_to_route(route2)
        service2.assign_to_route(route1)

        self._invalidate_cache()
        return True

    # Verifica se serviço está atribuído
    def is_service_assigned(self, service: Service) -> bool:
        for services in self.assignment.values():
            if service in services:
                return True
        return False

    # Retorna todos os serviços atribuídos
    def get_assigned_services(self) -> List[Service]:
        all_services = []
        for services in self.assignment.values():
            all_services.extend(services)
        return all_services

    # Retorna serviços não atribuídos
    def get_unassigned_services(self, all_services: List[Service]) -> List[Service]:
        assigned = set(self.get_assigned_services())
        return [s for s in all_services if s not in assigned]

    #Contagem de serviços atribuídos
    def count_assigned_services(self) -> int:
        return sum(len(services) for services in self.assignment.values())

    #Nível da capacidade de serviço de uma rota;
    def get_route_utilization(self, route: Route) -> float:

        if route.capacity == 0:
            return 0.0
        return len(self.assignment[route]) / route.capacity

    # Nível da capacidade de serviço de uma rota;
    def get_average_utilization(self) -> float:
        if not self.routes:
            return 0.0
        return sum(self.get_route_utilization(r) for r in self.routes) / len(self.routes)

    def total_distance(self) -> float:
        """Calcula distância total"""
        if self._total_distance is None:
            self._total_distance = sum(
                self._calculate_route_distance(route, services)
                for route, services in self.assignment.items()
            )
        return self._total_distance

    # Calcula distância de uma rota específica
    def _calculate_route_distance(self, route: Route, services: List[Service]) -> float:
        if not services:
            return 0.0

        total = 0.0

        # Base -> primeiro serviço
        dist_info = dist_entre_coordenadas(route.position, services[0].position)
        if dist_info and dist_info[0]:
            total += dist_info[0] / 1000

        # Entre serviços
        for i in range(len(services) - 1):
            dist_info = dist_entre_coordenadas(services[i].position, services[i + 1].position)
            if dist_info and dist_info[0]:
                total += dist_info[0] / 1000

        # Último -> base
        dist_info = dist_entre_coordenadas(services[-1].position, route.position)
        if dist_info and dist_info[0]:
            total += dist_info[0] / 1000

        return total

    def calculate_objective_function(self, all_services: List[Service],
                                    alpha=1.0, beta=100.0, gamma=50.0) -> float:
        """
        Função objetivo com três critérios:
        critérios:
            alpha: Peso da distância
            beta: Peso por serviço não atendido
            gamma: Peso adicional por serviço crítico não atendido
        """
        distance = self.total_distance()
        unassigned = self.get_unassigned_services(all_services)
        num_unassigned = len(unassigned)

        critical_cost = sum(
            s.calculate_cost()
            for s in unassigned
            if s.get_priority() in [ServicePriority.CRITICAL, ServicePriority.HIGH]
        )

        return alpha * distance + beta * num_unassigned + gamma * critical_cost


    # Verifica se solução é válida
    def is_valid(self, max_distance=50) -> bool:

        if self._is_valid is not None:
            return self._is_valid

        # Verifica capacidade
        for route, services in self.assignment.items():
            if len(services) > route.capacity:
                self._is_valid = False
                return False

        # Verifica duplicação
        all_services = []
        for services in self.assignment.values():
            all_services.extend(services)

        if len(all_services) != len(set(all_services)):
            self._is_valid = False
            return False

        # Verifica distâncias
        for route, services in self.assignment.items():
            for service in services:
                if not route.can_reach(service, max_distance):
                    self._is_valid = False
                    return False

        self._is_valid = True
        return True

    # Retorna lista de erros de validação
    def get_validation_errors(self, max_distance=50) -> List[str]:

        errors = []

        for route, services in self.assignment.items():
            if len(services) > route.capacity:
                errors.append(
                    f"Rota {route.route_id}: {len(services)} serviços excedem capacidade {route.capacity}"
                )

        seen = set()
        for services in self.assignment.values():
            for service in services:
                if service in seen:
                    errors.append(f"Serviço {service.service_id} atribuído múltiplas vezes")
                seen.add(service)

        for route, services in self.assignment.items():
            for service in services:
                if not route.can_reach(service, max_distance):
                    dist = route.distance_to(service)
                    errors.append(
                        f"Rota {route.route_id} não pode alcançar serviço {service.service_id} "
                        f"(distância: {dist:.2f} km > {max_distance} km)"
                    )

        return errors

      # Cria cópia profunda
    def copy(self):

        new_solution = Solution(self.routes)

        for route, services in self.assignment.items():
            new_solution.assignment[route] = services.copy()

        new_solution._total_distance = self._total_distance
        new_solution._total_cost = self._total_cost
        new_solution._is_valid = self._is_valid
        new_solution.iteration_found = self.iteration_found

        return new_solution

    # Invalida cache após modificações
    def _invalidate_cache(self):
        self._total_distance = None
        self._total_cost = None
        self._is_valid = None

    #Imprime resumo da solução
    def print_summary(self):

        print("=" * 60)
        print(f"SOLUÇÃO - Iteração {self.iteration_found}")
        print("=" * 60)
        print(f"Total de rotas: {len(self.routes)}")
        print(f"Serviços atribuídos: {self.count_assigned_services()}")
        print(f"Distância total: {self.total_distance():.2f} km")
        print(f"Utilização média: {self.get_average_utilization():.1%}")
        print(f"Válida: {'✓' if self.is_valid() else '✗'}")

        if not self.is_valid():
            errors = self.get_validation_errors()
            print(f"\nErros encontrados ({len(errors)}):")
            for error in errors[:5]:
                print(f"  - {error}")

        print("\nDetalhamento por rota:")
        print("-" * 60)
        for route, services in self.assignment.items():
            dist = self._calculate_route_distance(route, services)
            util = self.get_route_utilization(route)
            service_ids = [s.service_id for s in services]
            print(f"Rota {route.route_id}: {len(services)}/{route.capacity} serviços "
                  f"({util:.1%}) - {dist:.2f} km")
            print(f"  Serviços: {service_ids}")
        print("=" * 60)

    # Converte para dicionário
    def to_dict(self) -> Dict:
        return {
            'routes': [
                {
                    'route_id': route.route_id,
                    'position': route.position,
                    'capacity': route.capacity,
                    'services': [s.service_id for s in services]
                }
                for route, services in self.assignment.items()
            ],
            'metrics': {
                'total_distance': self.total_distance(),
                'num_services': self.count_assigned_services(),
                'average_utilization': self.get_average_utilization(),
                'is_valid': self.is_valid()
            },
            'timestamp': self.creation_time.isoformat(),
            'iteration': self.iteration_found
        }

In [ ]:
# ============================================================================
# ILS OTIMIZADO - VERSÃO COM MELHORIAS DE PERFORMANCE
# ============================================================================

# --- 1. CACHE DE DISTÂNCIAS (Ganho: 5-10x mais rápido) ---
class DistanceCache:
    """Cache global de distâncias calculadas para evitar recálculos"""
    def __init__(self):
        self._cache = {}
    
    def get_distance(self, coord1, coord2):
        """Retorna distância em km usando cache"""
        # Cria chave ordenada (A->B = B->A)
        key = tuple(sorted([coord1, coord2]))
        
        if key not in self._cache:
            dist_info = dist_entre_coordenadas(coord1, coord2)
            self._cache[key] = dist_info[0] / 1000 if dist_info and dist_info[0] else float('inf')
        
        return self._cache[key]
    
    def clear(self):
        self._cache.clear()
    
    def size(self):
        return len(self._cache)

# Instância global do cache
_distance_cache = DistanceCache()


# --- 2. SOLUÇÃO INICIAL GULOSA CORRIGIDA ---
def generate_initial_solution(routes: List[Route], services: List[Service], 
                              max_distance=50) -> Tuple[Solution, List[Service]]:
    """
    Solução inicial gulosa otimizada
    
    CORREÇÕES DOS BUGS:
    - Condição while corrigida: < 1 ao invés de >= 1
    - Filtro de distância corrigido: > max_distance ao invés de < 500
    
    Estratégia:
    1. Ordena serviços por prioridade (críticos primeiro)
    2. Para cada serviço, atribui à rota mais próxima com capacidade
    3. Usa cache de distâncias
    """
    solution = Solution(routes)
    unvisited = services.copy()
    unvisited.sort(key=lambda s: (s.get_priority().value, s.service_id))
    
    current_positions = {route: route.position for route in routes}
    
    # BUG CORRIGIDO: Era >= 1, agora é < 1
    while unvisited and (solution.get_average_utilization() < 1):
        best_route = None
        best_service = None
        best_distance = float("inf")
        
        for service in unvisited:
            for route in routes:
                if len(solution.assignment[route]) >= route.capacity:
                    continue
                
                # USA CACHE: mais rápido
                distance_km = _distance_cache.get_distance(
                    current_positions[route], 
                    service.position
                )
                
                if distance_km == float('inf'):
                    continue
                
                # BUG CORRIGIDO: Era < 500, agora é > max_distance
                if distance_km > max_distance:
                    continue
                
                if distance_km < best_distance:
                    best_route = route
                    best_service = service
                    best_distance = distance_km
        
        if best_route is None:
            log(f"  ⚠️ Não foi possível alocar {len(unvisited)} serviços restantes")
            break
        
        if solution.assign_service(best_route, best_service):
            current_positions[best_route] = best_service.position
            unvisited.remove(best_service)
            log(f"  ✓ S{best_service.service_id} → R{best_route.route_id} "
                f"({best_distance:.2f}km, {best_service.get_priority().name})")
        else:
            unvisited.remove(best_service)
    
    return solution, unvisited


# --- 3. BUSCA LOCAL COM FIRST IMPROVEMENT + 2-OPT ---
def local_search_optimized(solution: Solution, max_iter=100, 
                          strategy='first', use_2opt=True) -> Solution:
    """
    Busca local otimizada
    
    Args:
        strategy: 'first' (para ao encontrar melhoria, 3-5x mais rápido)
                 'best' (testa todas, melhor qualidade)
        use_2opt: Aplica 2-opt para otimizar ordem intra-rota
    
    Melhorias:
    - First improvement (para na primeira melhoria)
    - Usa cache de distâncias
    - 2-opt para otimizar ordem dos serviços dentro de cada rota
    """
    best = solution.copy()
    best_cost = best.total_distance()
    
    improved = True
    iteration = 0
    
    log(f"🔍 Busca local (estratégia: {strategy}, custo inicial: {best_cost:.2f}km)")
    
    while improved and iteration < max_iter:
        improved = False
        iteration += 1
        
        # --- RELOCATE: Move serviços entre rotas ---
        for route_from in best.routes:
            if improved and strategy == 'first':
                break
            
            services_from = best.assignment[route_from].copy()
            
            for route_to in best.routes:
                if improved and strategy == 'first':
                    break
                
                if route_from == route_to:
                    continue
                
                if not route_to.has_capacity_for(1):
                    continue
                
                for service in services_from:
                    # Verifica se pode alcançar
                    dist = _distance_cache.get_distance(route_to.position, service.position)
                    if dist > 50:  # max_distance
                        continue
                    
                    candidate = best.copy()
                    
                    if candidate.move_service(service, route_from, route_to):
                        candidate_cost = candidate.total_distance()
                        
                        if candidate_cost < best_cost - 0.01:  # Melhoria significativa
                            best = candidate
                            best_cost = candidate_cost
                            improved = True
                            log(f"  ✓ Move: S{service.service_id} R{route_from.route_id}→R{route_to.route_id} ({best_cost:.2f}km)")
                            
                            if strategy == 'first':
                                break
        
        # --- SWAP: Troca serviços entre rotas ---
        if not improved:
            routes_list = list(best.routes)
            
            for i, route1 in enumerate(routes_list):
                if improved and strategy == 'first':
                    break
                
                services1 = best.assignment[route1].copy()
                
                for j, route2 in enumerate(routes_list):
                    if improved and strategy == 'first':
                        break
                    
                    if i >= j:
                        continue
                    
                    services2 = best.assignment[route2].copy()
                    
                    for service1 in services1:
                        for service2 in services2:
                            # Verifica distâncias
                            dist1 = _distance_cache.get_distance(route1.position, service2.position)
                            dist2 = _distance_cache.get_distance(route2.position, service1.position)
                            
                            if dist1 > 50 or dist2 > 50:
                                continue
                            
                            candidate = best.copy()
                            
                            if candidate.swap_services(service1, route1, service2, route2):
                                candidate_cost = candidate.total_distance()
                                
                                if candidate_cost < best_cost - 0.01:
                                    best = candidate
                                    best_cost = candidate_cost
                                    improved = True
                                    log(f"  ✓ Swap: S{service1.service_id}↔S{service2.service_id} ({best_cost:.2f}km)")
                                    
                                    if strategy == 'first':
                                        break
                        
                        if improved and strategy == 'first':
                            break
        
        # --- 2-OPT: Otimiza ordem dentro de cada rota ---
        if not improved and use_2opt:
            for route in best.routes:
                services = best.assignment[route]
                
                if len(services) < 4:
                    continue
                
                improved_order = _two_opt_route(route, services)
                
                if improved_order:
                    best.assignment[route] = improved_order
                    best._invalidate_cache()
                    new_cost = best.total_distance()
                    
                    if new_cost < best_cost:
                        best_cost = new_cost
                        improved = True
                        log(f"  ✓ 2-opt em R{route.route_id} ({best_cost:.2f}km)")
                        
                        if strategy == 'first':
                            break
    
    log(f"✓ Busca local: {iteration} iterações, custo final: {best_cost:.2f}km")
    return best


def _two_opt_route(route, services):
    """
    Aplica 2-opt para melhorar ordem dos serviços na rota
    Reverte segmentos para eliminar cruzamentos
    """
    best_order = services.copy()
    best_dist = _calculate_route_dist_cached(route, best_order)
    improved = False
    
    for i in range(len(services) - 1):
        for j in range(i + 2, len(services)):
            # Reverte segmento [i+1:j+1]
            new_order = services[:i+1] + services[i+1:j+1][::-1] + services[j+1:]
            new_dist = _calculate_route_dist_cached(route, new_order)
            
            if new_dist < best_dist - 0.01:
                best_order = new_order
                best_dist = new_dist
                improved = True
    
    return best_order if improved else None


def _calculate_route_dist_cached(route, services_list):
    """Calcula distância de rota usando cache"""
    if not services_list:
        return 0.0
    
    total = _distance_cache.get_distance(route.position, services_list[0].position)
    
    for i in range(len(services_list) - 1):
        total += _distance_cache.get_distance(
            services_list[i].position, 
            services_list[i+1].position
        )
    
    total += _distance_cache.get_distance(services_list[-1].position, route.position)
    return total


# --- 4. PERTURBAÇÃO INTELIGENTE ---
def perturb_smart(solution: Solution, remove_count=2, all_services=None):
    """
    Perturbação estratégica focada em rotas problemáticas
    
    Melhorias:
    - Remove serviços de rotas com pior custo/utilização
    - Prioriza remover serviços de baixa prioridade
    - Realoca usando heurística gulosa
    """
    perturbed = solution.copy()
    
    # Identifica rotas com pior desempenho
    route_scores = []
    for route in perturbed.routes:
        services = perturbed.assignment[route]
        if not services:
            continue
        
        dist = perturbed._calculate_route_distance(route, services)
        util = perturbed.get_route_utilization(route)
        
        # Score: rotas longas com baixa utilização são piores
        score = dist / max(util, 0.1)
        route_scores.append((score, route, services))
    
    if not route_scores:
        return perturbed
    
    # Ordena por score (piores primeiro)
    route_scores.sort(reverse=True, key=lambda x: x[0])
    
    removed_services = []
    removed_count = 0
    
    # Remove serviços das rotas problemáticas
    for score, route, services in route_scores:
        if removed_count >= remove_count:
            break
        
        # Prioriza remover serviços de baixa prioridade
        candidates = sorted(services, 
                          key=lambda s: s.get_priority().value, 
                          reverse=True)
        
        for service in candidates[:min(2, len(candidates))]:
            if removed_count >= remove_count:
                break
            
            perturbed.assignment[route].remove(service)
            service.unassign()
            removed_services.append(service)
            removed_count += 1
            log(f"  ✗ S{service.service_id} removido de R{route.route_id}")
    
    perturbed._invalidate_cache()
    
    # Realoca usando inserção gulosa
    removed_services.sort(key=lambda s: s.get_priority().value)
    
    for service in removed_services:
        best_route = None
        best_cost = float('inf')
        
        for route in perturbed.routes:
            if not route.has_capacity_for(1):
                continue
            
            dist = _distance_cache.get_distance(route.position, service.position)
            if dist > 50:
                continue
            
            # Custo de inserção
            current_services = perturbed.assignment[route]
            if not current_services:
                cost = 2 * dist
            else:
                # Insere no final (simplificação)
                cost = _distance_cache.get_distance(current_services[-1].position, service.position)
                cost += _distance_cache.get_distance(service.position, route.position)
                cost -= _distance_cache.get_distance(current_services[-1].position, route.position)
            
            if cost < best_cost:
                best_route = route
                best_cost = cost
        
        if best_route:
            perturbed.assign_service(best_route, service)
            log(f"  ✓ S{service.service_id} → R{best_route.route_id}")
    
    return perturbed


# --- 5. ILS OTIMIZADO COM SIMULATED ANNEALING ---
def ils_optimized(routes: List[Route], services: List[Service],
                 iterations=100, seed=123, perturb_size=2,
                 temp_initial=100.0, temp_decay=0.95,
                 use_smart_perturb=True, use_2opt=True) -> Solution:
    """
    ILS OTIMIZADO com todas as melhorias:
    
    1. ✅ Cache de distâncias (5-10x mais rápido)
    2. ✅ Solução inicial gulosa corrigida
    3. ✅ First improvement na busca local (3-5x mais rápido)
    4. ✅ Simulated Annealing (escapa ótimos locais)
    5. ✅ Perturbação inteligente
    6. ✅ 2-opt intra-rota
    7. ✅ Restart adaptativo
    
    Args:
        temp_initial: Temperatura inicial para SA (maior = mais exploração)
        temp_decay: Taxa de resfriamento (0.9-0.99)
        use_smart_perturb: Usa perturbação inteligente ao invés de aleatória
        use_2opt: Aplica 2-opt na busca local
    """
    random.seed(seed)
    _distance_cache.clear()
    
    log("🚀 ILS OTIMIZADO - Gerando solução inicial...")
    current, unvisited = generate_initial_solution(routes, services)
    
    log(f"✓ Solução inicial: {current.count_assigned_services()}/{len(services)} serviços, "
        f"{current.total_distance():.2f}km")
    
    if unvisited:
        log(f"⚠️  {len(unvisited)} serviços não alocados inicialmente")
    
    log("🔍 Aplicando busca local inicial...")
    current = local_search_optimized(current, strategy='first', use_2opt=use_2opt)
    
    best = current.copy()
    best.iteration_found = 0
    best_cost = best.calculate_objective_function(services)
    
    log(f"✓ Solução inicial otimizada: custo={best_cost:.2f}\n")
    log(f"🔄 Iniciando ILS: {iterations} iterações, temp={temp_initial}, decay={temp_decay}\n")
    
    temperature = temp_initial
    no_improvement_count = 0
    
    for it in range(iterations):
        # Perturbação adaptativa
        adaptive_size = perturb_size + (1 if no_improvement_count > 10 else 0)
        
        if use_smart_perturb:
            candidate = perturb_smart(current, remove_count=adaptive_size, 
                                     all_services=services)
        else:
            candidate = perturb(current, remove_count=adaptive_size)
        
        # Busca local rápida
        candidate = local_search_optimized(candidate, max_iter=50, 
                                          strategy='first', use_2opt=use_2opt)
        
        # Custos
        candidate_cost = candidate.calculate_objective_function(services)
        current_cost = current.calculate_objective_function(services)
        
        # Critério de aceitação com Simulated Annealing
        delta = candidate_cost - current_cost
        
        if delta < 0:
            # Melhorou: sempre aceita
            current = candidate
            no_improvement_count = 0
        else:
            # Piorou: aceita com probabilidade baseada em temperatura
            accept_prob = math.exp(-delta / temperature) if temperature > 0 else 0
            
            if random.random() < accept_prob:
                current = candidate
                log(f"[Iter {it+1}] 🔥 Aceitou pior (Δ={delta:.2f}, p={accept_prob:.3f})")
            else:
                no_improvement_count += 1
        
        # Atualiza melhor
        if candidate_cost < best_cost:
            best = candidate.copy()
            best.iteration_found = it + 1
            best_cost = candidate_cost
            no_improvement_count = 0
            
            log(f"[Iter {it+1}] 🌟 NOVA MELHOR!")
            log(f"  Custo: {best_cost:.2f}")
            log(f"  Distância: {best.total_distance():.2f}km")
            log(f"  Serviços: {best.count_assigned_services()}/{len(services)}")
        
        # Resfriamento
        temperature *= temp_decay
        
        # Restart se estagnado
        if no_improvement_count > 20:
            log(f"[Iter {it+1}] 🔄 RESTART (estagnado há {no_improvement_count} iterações)")
            current = perturb_smart(best, remove_count=perturb_size * 2) if use_smart_perturb else perturb(best, remove_count=perturb_size * 2)
            no_improvement_count = 0
            temperature = temp_initial * 0.5
        
        # Log periódico
        if (it + 1) % 20 == 0:
            log(f"[Iter {it+1}] temp={temperature:.2f}, cache={_distance_cache.size()}, melhor={best_cost:.2f}")
    
    log(f"\n✅ ILS finalizado!")
    log(f"Melhor solução na iteração {best.iteration_found}")
    log(f"Cache de distâncias: {_distance_cache.size()} entradas")
    
    return best


# Mantém compatibilidade com código antigo
def local_search(solution: Solution, max_iter=100) -> Solution:
    """Wrapper para compatibilidade"""
    return local_search_optimized(solution, max_iter=max_iter, strategy='first', use_2opt=True)


def perturb(solution: Solution, remove_count=1) -> Solution:
    """Perturbação aleatória original (mantida para compatibilidade)"""
    perturbed = solution.copy()
    
    assigned_pairs = []
    for route in perturbed.routes:
        for service in perturbed.assignment[route]:
            assigned_pairs.append((route, service))
    
    if not assigned_pairs:
        return perturbed
    
    actual_remove_count = min(remove_count, len(assigned_pairs))
    removed_services = []
    
    for _ in range(actual_remove_count):
        if not assigned_pairs:
            break
        
        route_from, service = random.choice(assigned_pairs)
        
        if service in perturbed.assignment[route_from]:
            perturbed.assignment[route_from].remove(service)
            service.unassign()
            removed_services.append(service)
            assigned_pairs = [(r, s) for r, s in assigned_pairs if s != service]
    
    perturbed._invalidate_cache()
    
    # Realoca
    for service in removed_services:
        candidates = [
            route for route in perturbed.routes
            if route.has_capacity_for(1) and len(perturbed.assignment[route]) < route.capacity
        ]
        
        if candidates:
            # Usa cache para escolher melhor rota
            best_route = min(candidates, 
                           key=lambda r: _distance_cache.get_distance(r.position, service.position))
            perturbed.assign_service(best_route, service)
    
    return perturbed


def ils(routes: List[Route], services: List[Service],
        iterations=100, seed=123, perturb_size=1) -> Solution:
    """Wrapper para compatibilidade - chama versão otimizada"""
    return ils_optimized(
        routes=routes,
        services=services,
        iterations=iterations,
        seed=seed,
        perturb_size=perturb_size,
        temp_initial=100.0,
        temp_decay=0.95,
        use_smart_perturb=True,
        use_2opt=True
    )

In [9]:
def ils(routes: List[Route], services: List[Service],
        iterations=100, seed=123, perturb_size=1) -> Solution:
    """
    Iterated Local Search

    Args:
        routes: Lista de rotas disponíveis
        services: Lista de serviços
        iterations: Número de iterações
        seed: Semente aleatória
        perturb_size: Tamanho da perturbação

    Returns:
        Melhor solução encontrada
    """
    random.seed(seed)

    log("🚀 Gerando solução inicial gulosa...")
    current, unvisited = generate_initial_solution(routes, services)

    log(f"✓ Solução inicial: {current.count_assigned_services()}/{len(services)} serviços, "
        f"{current.total_distance():.2f} km")

    if unvisited:
        log(f"⚠️  {len(unvisited)} serviços não alocados na solução inicial")

    log("🔍 Aplicando busca local inicial...")
    current = local_search(current)

    best = current.copy()
    best.iteration_found = 0
    best_cost = best.calculate_objective_function(services)

    log(f"✓ Solução inicial otimizada: custo={best_cost:.2f}")

    log(f"\n🔄 Iniciando ILS com {iterations} iterações...")

    for it in range(iterations):
        # Perturba
        candidate = perturb(current, remove_count=perturb_size)

        # Busca local
        candidate = local_search(candidate)

        # Calcula custos
        candidate_cost = candidate.calculate_objective_function(services)
        current_cost = current.calculate_objective_function(services)

        # Critério de aceitação
        if candidate_cost < current_cost:
            current = candidate

        # Atualiza melhor solução
        if candidate_cost < best_cost:
            best = candidate.copy()
            best.iteration_found = it + 1
            best_cost = candidate_cost

            log(f"[Iter {it+1}] 🌟 Nova melhor solução!")
            log(f"  Custo: {best_cost:.2f}")
            log(f"  Distância: {best.total_distance():.2f} km")
            log(f"  Serviços: {best.count_assigned_services()}/{len(services)}")

        # Log periódico
        if (it + 1) % 20 == 0:
            log(f"[Iter {it+1}] Progresso: custo atual={current_cost:.2f}, melhor={best_cost:.2f}")

    log(f"\n✅ ILS finalizado!")
    log(f"Melhor solução encontrada na iteração {best.iteration_found}")

    return best


In [10]:

# ============================================================================
# VISUALIZAÇÃO
# ============================================================================

def plotar_rotas_simples(solution: Solution, all_services: List[Service],
                        salvar_como='rotas_mapa.html') -> folium.Map:
    """
    Versão simplificada da plotagem (sem OSMnx para evitar erros)
    """
    log("📍 Gerando mapa das rotas...")

    # Calcula centro do mapa
    all_points = []
    for route in solution.routes:
        all_points.append(route.position)
        for service in solution.assignment[route]:
            all_points.append(service.position)

    if all_points:
        center_lat = sum(p[0] for p in all_points) / len(all_points)
        center_lon = sum(p[1] for p in all_points) / len(all_points)
    else:
        center_lat, center_lon = -9.6658, -35.7350  # Maceió centro

    # Cria mapa
    mapa = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles='OpenStreetMap'
    )

    # Cores por rota
    cores = ['red', 'blue', 'green', 'purple', 'orange', 'darkred']

    # Cores por prioridade
    cores_prioridade = {
        'CRITICAL': '#FF0000',
        'HIGH': '#FF6600',
        'MEDIUM': '#FFCC00',
        'LOW': '#00CC00'
    }

    # Plota cada rota
    for idx, route in enumerate(solution.routes):
        services = solution.assignment[route]
        cor = cores[idx % len(cores)]

        # Grupo para a rota
        grupo = folium.FeatureGroup(
            name=f"Rota {route.route_id} ({len(services)}/{route.capacity})",
            show=True
        )

        # Marcador da base
        folium.Marker(
            location=route.position,
            popup=f"<b>Base - Rota {route.route_id}</b><br>Capacidade: {route.capacity}",
            tooltip=f"Base Rota {route.route_id}",
            icon=folium.Icon(color=cor, icon='home', prefix='fa')
        ).add_to(grupo)

        # Linha da rota (simplificada - linha reta)
        if services:
            pontos = [route.position]
            for service in services:
                pontos.append(service.position)
            pontos.append(route.position)  # Retorna à base

            folium.PolyLine(
                locations=pontos,
                color=cor,
                weight=4,
                opacity=0.7,
                tooltip=f"Rota {route.route_id}"
            ).add_to(grupo)

        # Marcadores dos serviços
        for i, service in enumerate(services):
            cor_prior = cores_prioridade.get(service.get_priority().name, '#808080')

            popup_html = f"""
            <b>Serviço {service.service_id}</b><br>
            Tipo: {service.service_type.value}<br>
            Prioridade: {service.get_priority().name}<br>
            Ordem: {i+1}º na rota
            """

            folium.Marker(
                location=service.position,
                popup=popup_html,
                tooltip=f"S{service.service_id}",
                icon=folium.Icon(color=cor, icon='info-sign')
            ).add_to(grupo)

            # Círculo com cor da prioridade
            folium.Circle(
                location=service.position,
                radius=100,
                color=cor_prior,
                fill=True,
                fillColor=cor_prior,
                fillOpacity=0.3
            ).add_to(grupo)

        grupo.add_to(mapa)

    # Serviços não atendidos
    unassigned = solution.get_unassigned_services(all_services)
    if unassigned:
        grupo_nao = folium.FeatureGroup(
            name=f"⚠️ Não Atendidos ({len(unassigned)})",
            show=True
        )

        for service in unassigned:
            folium.Marker(
                location=service.position,
                popup=f"<b>⚠️ NÃO ATENDIDO</b><br>Serviço {service.service_id}<br>{service.service_type.value}",
                tooltip=f"⚠️ S{service.service_id}",
                icon=folium.Icon(color='gray', icon='times', prefix='fa')
            ).add_to(grupo_nao)

        grupo_nao.add_to(mapa)

    # Adiciona controle de camadas
    folium.LayerControl().add_to(mapa)

    # Adiciona painel de métricas
    metricas_html = f"""
    <div style="position: fixed;
                top: 10px; right: 10px; width: 300px;
                background-color: white;
                border: 2px solid grey;
                border-radius: 5px;
                z-index: 9999;
                padding: 10px;">
        <h4>📊 Métricas</h4>
        <p><b>Rotas:</b> {len(solution.routes)}</p>
        <p><b>Serviços:</b> {solution.count_assigned_services()}/{len(all_services)}</p>
        <p><b>Taxa:</b> {solution.count_assigned_services()/len(all_services)*100:.1f}%</p>
        <p><b>Distância:</b> {solution.total_distance():.2f} km</p>
        <p><b>Utilização:</b> {solution.get_average_utilization():.1%}</p>
    </div>
    """
    mapa.get_root().html.add_child(folium.Element(metricas_html))

    # Salva
    if salvar_como:
        mapa.save(salvar_como)
        log(f"✅ Mapa salvo em: {salvar_como}")

    return mapa

def exportar_relatorio(solution: Solution, all_services: List[Service],
                      filename='relatorio_rotas.txt'):
    """Exporta relatório textual detalhado"""
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("RELATÓRIO DE ROTEIRIZAÇÃO - ALAGOAS\n")
        f.write("="*80 + "\n\n")

        f.write(f"Data: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
        f.write(f"Iteração da melhor solução: {solution.iteration_found}\n\n")

        f.write("-"*80 + "\n")
        f.write("MÉTRICAS GLOBAIS\n")
        f.write("-"*80 + "\n")
        f.write(f"Total de Rotas: {len(solution.routes)}\n")
        f.write(f"Total de Serviços: {len(all_services)}\n")
        f.write(f"Serviços Atendidos: {solution.count_assigned_services()}\n")
        f.write(f"Serviços Não Atendidos: {len(solution.get_unassigned_services(all_services))}\n")
        f.write(f"Taxa de Atendimento: {solution.count_assigned_services()/len(all_services)*100:.1f}%\n")
        f.write(f"Distância Total: {solution.total_distance():.2f} km\n")
        f.write(f"Utilização Média: {solution.get_average_utilization():.1%}\n")
        f.write(f"Válida: {'Sim' if solution.is_valid() else 'Não'}\n\n")

        f.write("="*80 + "\n")
        f.write("DETALHAMENTO POR ROTA\n")
        f.write("="*80 + "\n\n")

        for route in solution.routes:
            services = solution.assignment[route]
            dist = solution._calculate_route_distance(route, services)
            util = solution.get_route_utilization(route)

            f.write(f"ROTA {route.route_id}\n")
            f.write("-"*80 + "\n")
            f.write(f"Base: ({route.position[0]:.6f}, {route.position[1]:.6f})\n")
            f.write(f"Capacidade: {len(services)}/{route.capacity} ({util:.1%})\n")
            f.write(f"Distância: {dist:.2f} km\n\n")

            if services:
                f.write("Serviços:\n")
                for i, service in enumerate(services, 1):
                    f.write(f"  {i}. S{service.service_id} - {service.service_type.value} "
                           f"[{service.get_priority().name}]\n")
                    f.write(f"     ({service.position[0]:.6f}, {service.position[1]:.6f})\n")

            f.write("\n")

        unassigned = solution.get_unassigned_services(all_services)
        if unassigned:
            f.write("="*80 + "\n")
            f.write(f"SERVIÇOS NÃO ATENDIDOS ({len(unassigned)})\n")
            f.write("="*80 + "\n")
            for service in unassigned:
                f.write(f"  S{service.service_id} - {service.service_type.value} "
                       f"[{service.get_priority().name}]\n")

    log(f"✅ Relatório exportado: {filename}")



In [14]:
# ============================================================================
# DADOS DE EXEMPLO - ALAGOAS
# ============================================================================

def criar_exemplo_alagoas():
    """
    Cria exemplo completo para o estado de Alagoas

    Localizações reais:
    - Maceió (capital)
    - Arapiraca (2ª maior cidade)
    - Palmeira dos Índios
    - União dos Palmares
    - Rio Largo
    - Satuba
    - Marechal Deodoro
    - Pilar
    """

    log("\n" + "="*80)
    log("INICIANDO SISTEMA DE ROTEIRIZAÇÃO - ALAGOAS")
    log("="*80 + "\n")

    # Define bases das equipes (rotas)
    bases = [
        (-9.637, -35.724),   # Maceió Centro
        (-9.7525, -36.6612),   # Arapiraca
        (-9.4056, -36.6289),   # Palmeira dos Índios
    ]

    routes = [
        Route(route_id=0, position=bases[0], capacity=8),   # Equipe Maceió
        Route(route_id=1, position=bases[1], capacity=6),   # Equipe Arapiraca
        Route(route_id=2, position=bases[2], capacity=5),   # Equipe Palmeira
    ]

    log(f"✓ {len(routes)} equipes criadas:")
    for r in routes:
        log(f"  - Rota {r.route_id}: Base {r.position}, Capacidade {r.capacity}")
  # (, -35.7350),   # Maceió - Centro
    # Define locais de serviços (coordenadas reais em Alagoas)
    servicos_coords = [
        # Região Metropolitana de Maceió

        (-9.6400, -35.7100),   # Maceió - Ponta Verde
        (-9.6850, -35.7050),   # Maceió - Pajuçara
        (-9.5500, -35.7800),   # Rio Largo
        (-9.6200, -35.6900),   # Satuba
        (-9.7100, -35.8950),   # Marechal Deodoro
        (-9.6000, -35.8500),   # Pilar

        # Região de Arapiraca
        (-9.7525, -36.6612),   # Arapiraca - Centro
        (-9.7300, -36.6500),   # Arapiraca - Bairro Primavera
        (-9.7700, -36.6800),   # Arapiraca - Baixão
        (-9.7500, -36.5000),   # Craíbas
        (-9.8000, -36.7500),   # Lagoa da Canoa

        # Região Agreste
        (-9.4056, -36.6289),   # Palmeira dos Índios
        (-9.1500, -36.6800),   # União dos Palmares
        (-9.2800, -36.5500),   # Branquinha
        (-9.5200, -36.4500),   # Atalaia
        (-9.3500, -36.8000),   # Viçosa
        (-9.4800, -36.7200),   # Mar Vermelho
        (-9.1000, -36.5200),   # Murici
    ]

    # Cria serviços com tipos variados
    services = []

    for i, coord in enumerate(servicos_coords):
        service = Service(
            service_id=i,
            position=coord,
            duration=random.randint(20, 60)
        )

        # Define tipo e prioridade de forma variada
        tipo_rand = i % 3

        if tipo_rand == 0:  # Emergencial
            service.set_emergency_data(
                penalty=random.randint(3000, 8000),
                coverage=f"Região {i}",
                waiting_time=random.randint(30, 180),
                affected_customers=random.randint(50, 200)
            )

        elif tipo_rand == 1:  # Suspensão
            service.set_suspension_data(
                num_invoices=random.randint(3, 10),
                total_debt=random.randint(3000, 15000)
            )

        else:  # Regulada
            service.set_regulated_data(
                deadline=random.randint(1, 15),
                compensation_penalty=random.randint(100, 500)
            )

        services.append(service)

    log(f"\n✓ {len(services)} serviços criados:")

    # Conta por tipo
    tipos_count = {}
    prioridades_count = {}

    for s in services:
        tipo = s.service_type.value
        prior = s.get_priority().name
        tipos_count[tipo] = tipos_count.get(tipo, 0) + 1
        prioridades_count[prior] = prioridades_count.get(prior, 0) + 1

    log("\n  Distribuição por tipo:")
    for tipo, count in sorted(tipos_count.items()):
        log(f"    - {tipo}: {count}")

    log("\n  Distribuição por prioridade:")
    for prior, count in sorted(prioridades_count.items()):
        log(f"    - {prior}: {count}")

    return routes, services


In [16]:

# ============================================================================
# EXECUÇÃO PRINCIPAL
# ============================================================================

def main():
    """Função principal de execução"""

    # Cria dados de exemplo
    routes, services = criar_exemplo_alagoas()

    # Executa ILS
    log("\n" + "="*80)
    log("EXECUTANDO ALGORITMO ILS")
    log("="*80 + "\n")

    melhor_solucao = ils(
        routes=routes,
        services=services,
        iterations=50,
        seed=42,
        perturb_size=2
    )

    # Exibe resumo
    log("\n" + "="*80)
    log("RESULTADOS FINAIS")
    log("="*80 + "\n")

    melhor_solucao.print_summary()

    # Estatísticas detalhadas
    log("\n📊 ESTATÍSTICAS DETALHADAS:\n")

    total_servicos = len(services)
    atendidos = melhor_solucao.count_assigned_services()
    nao_atendidos = total_servicos - atendidos

    log(f"Total de serviços: {total_servicos}")
    log(f"Atendidos: {atendidos} ({atendidos/total_servicos*100:.1f}%)")
    log(f"Não atendidos: {nao_atendidos}")
    log(f"Distância total: {melhor_solucao.total_distance():.2f} km")
    log(f"Utilização média das equipes: {melhor_solucao.get_average_utilization():.1%}")
    log(f"Função objetivo: {melhor_solucao.calculate_objective_function(services):.2f}")

    # Verifica serviços críticos não atendidos
    unassigned = melhor_solucao.get_unassigned_services(services)
    criticos_nao_atendidos = [
        s for s in unassigned
        if s.get_priority() in [ServicePriority.CRITICAL, ServicePriority.HIGH]
    ]

    if criticos_nao_atendidos:
        log(f"\n⚠️  ATENÇÃO: {len(criticos_nao_atendidos)} serviços CRÍTICOS/ALTOS não atendidos:")
        for s in criticos_nao_atendidos:
            log(f"  - Serviço {s.service_id}: {s.service_type.value} [{s.get_priority().name}]")

    # Gera visualizações
    log("\n" + "="*80)
    log("GERANDO VISUALIZAÇÕES")
    log("="*80 + "\n")

    try:
        mapa = plotar_rotas_simples(
            melhor_solucao,
            services,
            salvar_como='rotas_alagoas.html'
        )
        log("✅ Mapa interativo gerado com sucesso!")
    except Exception as e:
        log(f"⚠️  Erro ao gerar mapa: {e}")

    # Exporta relatório
    try:
        exportar_relatorio(
            melhor_solucao,
            services,
            filename='relatorio_alagoas.txt'
        )
        log("✅ Relatório textual gerado com sucesso!")
    except Exception as e:
        log(f"⚠️  Erro ao gerar relatório: {e}")

    # Exporta JSON
    try:
        with open('solucao_alagoas.json', 'w', encoding='utf-8') as f:
            json.dump(melhor_solucao.to_dict(), f, indent=2, ensure_ascii=False)
        log("✅ Dados JSON exportados com sucesso!")
    except Exception as e:
        log(f"⚠️  Erro ao exportar JSON: {e}")

    log("\n" + "="*80)
    log("EXECUÇÃO FINALIZADA")
    log("="*80)
    log("\nArquivos gerados:")
    log("  - rotas_alagoas.html (mapa interativo)")
    log("  - relatorio_alagoas.txt (relatório detalhado)")
    log("  - solucao_alagoas.json (dados estruturados)")
    log("  - rotas.log (log de execução)")
    log("\nAbra 'rotas_alagoas.html' no navegador para visualizar as rotas!\n")

# ============================================================================
# PONTO DE ENTRADA
# ============================================================================

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        log("\n\n⚠️  Execução interrompida pelo usuário")
    except Exception as e:
        log(f"\n\n❌ ERRO FATAL: {e}")
        import traceback
        log(traceback.format_exc())


INICIANDO SISTEMA DE ROTEIRIZAÇÃO - ALAGOAS

✓ 3 equipes criadas:
  - Rota 0: Base (-9.637, -35.724), Capacidade 8
  - Rota 1: Base (-9.7525, -36.6612), Capacidade 6
  - Rota 2: Base (-9.4056, -36.6289), Capacidade 5

✓ 18 serviços criados:

  Distribuição por tipo:
    - Emergencial: 6
    - Regulada: 6
    - Suspensão de Fornecimento: 6

  Distribuição por prioridade:
    - CRITICAL: 6
    - HIGH: 8
    - LOW: 3
    - MEDIUM: 1

EXECUTANDO ALGORITMO ILS

🚀 Gerando solução inicial gulosa...
✓ Solução inicial: 0/18 serviços, 0.00 km
⚠️  18 serviços não alocados na solução inicial
🔍 Aplicando busca local inicial...
🔍 Iniciando busca local (custo inicial: 0.00 km)
✓ Busca local concluída: 1 iterações, custo final: 0.00 km
✓ Solução inicial otimizada: custo=4612500.00

🔄 Iniciando ILS com 50 iterações...
🔍 Iniciando busca local (custo inicial: 0.00 km)
✓ Busca local concluída: 1 iterações, custo final: 0.00 km
🔍 Iniciando busca local (custo inicial: 0.00 km)
✓ Busca local concluída: 1 it